<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/train/generate_knEmbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test

In [ ]:
!rm -r /content/data/knowledge_embeddings

In [ ]:
!cp -r /content/data/knowledge_embeddings /content/drive/MyDrive/data

In [ ]:
!cp -r /content/data/knowledge_embeddings/. /content/drive/MyDrive/data/knowledge_embeddings

In [ ]:
!ls /content/data/knowledge_embeddings | wc -l

25762


In [ ]:
!ls /content/data/knowledge_embeddings/7006.npy

ls: cannot access '/content/data/knowledge_embeddings/7006.npy': No such file or directory


In [ ]:
!ls /content/drive/MyDrive/data/knowledge_embeddings | wc -l

32761


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

# Download and unzip data

In [ ]:
!cp /content/drive/MyDrive/files/data_path.zip /content
!unzip data_path.zip "data/jsons/*" "data/pic/*" "data/ques_embeddings/bioelmo/*" "data/answer_word_frequency.csv" "data/answer_word_frequency.xlsx" -d /content

In [ ]:
!mkdir data
!cp /content/drive/MyDrive/data/jsons/QA.json /content/data
!cp /content/drive/MyDrive/data/knowledge.txt /content/data

mkdir: cannot create directory ‘data’: File exists


# Object detection (暂时没用)

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# test installation
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
!cp /content/models/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb /content/drive/MyDrive/Colab\ Notebooks

# Entity extraction

## test

In [ ]:
# The textual entities are KG
# entities associated with key phrases of the question, where we utilize the Stanford NLP Dependency Parser [4] and Stanford Named
# Entity Recognizer [5] to analyze the question and extract all noun
# phrases, verb-object phrases and named entities as the key phrases.


In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import pathlib

root = pathlib.Path('/content/data')
d = shuffle(pd.read_json(root.joinpath('jsons/open_ended').with_suffix('.json')), random_state=1)

In [ ]:
dq = d['Questions']
print(dq)

12134                                     What is present?
9969                                      What is present?
3785                                     What is present ?
2947                            What does this image show?
858                     What are colors of these hormones?
                               ...                        
7813                                      What is present?
10955                           What does this image show?
5192                            Where does this belong to?
12172                                     What is present?
235      How many copies does the red probe hybridize t...
Name: Questions, Length: 16429, dtype: object


In [ ]:
print(dq[0])

What are positively charged,  thus allowing the compaction of the negatively charged DNA?


In [ ]:
pip install stanfordcorenlp

## Extraction Method

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def ectract_query(ques):
  qs = nlp(ques)
  queries = []
  for tk in qs:
    # print(tk.text,'|',tk.pos_)
    if tk.pos_ == 'VERB' or tk.pos_ == 'NOUN' or tk.pos_ == 'ADJ':
      queries.append(tk.lemma_)
  return queries


# print(ectract_query(dq[0]))

In [ ]:
def knowledge_retrieval(queries):
  kns = []
  for q in queries:
    kns += query_knowledge(q)
  return(kns)

# queries = ectract_query(dq[0])
# query_knowledge

# Knowledge Embeddings

In [ ]:
pip install transformers

     |████████████████████████████████| 2.1MB 4.4MB/s 
     |████████████████████████████████| 870kB 11.8MB/s 
     |████████████████████████████████| 3.3MB 26.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=aaeac8c7a3bb3ca4bf7f4600a082464996d2eebe405aac07c85caf32594dc485
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import json
import re
import os

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased", do_lower_case=True)
model = AutoModel.from_pretrained("bert-large-cased", output_hidden_states=True)

with open('/content/data/knowledge.txt','r') as file:
    lines = file.readlines()


# def preprocess_sentence(w):
#     w = w.lower().strip()
#     w = re.sub(r'[" "]+', " ", w)
#     w = re.sub(r"[^a-zA-Z ]", "", w)
#     w = w.strip()
#     return w


path = '/content/data/knowledge_embeddings'
if not os.path.exists(path):
    os.mkdir(path)


In [ ]:
kns = []
for l in lines:
  l_split = l.strip().split(',')
  kns.append((l_split[0],l_split[1:]))
print(len(kns))

32761


In [ ]:
print(kns[6998])

('7006', ['auditory image is a image', 'show used for entertain', 'radical mastectomy is a mastectomy', 'scar at location finger', 'skin used for covering body', 'metastasis is a pathologic process'])


In [ ]:
for kn in kns:
  ques_id = int(kn[0])
  ks = kn[1]
  k_emb = []
  for k in ks:
    k = '[CLS] ' + k + ' [SEP]'
    tokenized_text1 = tokenizer.tokenize(k) 
    indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)    
    segments_ids1 = [1] * len(tokenized_text1)  
    # shape: (1,17)  
    tokens_tensor1 = torch.tensor([indexed_tokens1])
    segments_tensors1 = torch.tensor([segments_ids1])
    with torch.no_grad():
      # Because we set 'output_hidden_states = True', the third item will be the
      # hidden states from all layers.
      outputs1 = model(tokens_tensor1, segments_tensors1)
      hidden_states1 = outputs1[2]
    
    # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
    token_embeddings1 = torch.stack(hidden_states1, dim=0)
    # Remove dimension 1, the "batches".
    # shape: (25,17,1024) - (layers,sentence length,embedding size)
    token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
    # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
    # shape: (17,25,1024) - (sentence length,layers,embedding size)
    token_embeddings1 = token_embeddings1.permute(1, 0, 2)
    token_vecs_sum1 = []

    # shape: (17,1024)
    for token in token_embeddings1:
        # Sum the vectors from the last four layers.
        sum_vec1 = torch.sum(token[-4:], dim=0)
        token_vecs_sum1.append(sum_vec1)
    
    
    k_emb += token_vecs_sum1
  # knowledge embedding for one question
  # shape: (27,1024)
  k_emb = np.array([t.numpy() for t in k_emb])
  # save
  np.save(os.path.join(path, str(ques_id)+'.npy'), k_emb)
  print("saved file: ", ques_id)
print("saved all")

流式输出内容被截断，只能显示最后 5000 行内容。
saved file:  27787
saved file:  27788
saved file:  27789
saved file:  27790
saved file:  27791
saved file:  27792
saved file:  27793
saved file:  27794
saved file:  27795
saved file:  27796
saved file:  27797
saved file:  27798
saved file:  27799
saved file:  27800
saved file:  27801
saved file:  27802
saved file:  27803
saved file:  27804
saved file:  27805
saved file:  27806
saved file:  27807
saved file:  27808
saved file:  27809
saved file:  27810
saved file:  27811
saved file:  27812
saved file:  27813
saved file:  27814
saved file:  27815
saved file:  27816
saved file:  27817
saved file:  27818
saved file:  27819
saved file:  27820
saved file:  27821
saved file:  27822
saved file:  27823
saved file:  27824
saved file:  27825
saved file:  27826
saved file:  27827
saved file:  27828
saved file:  27829
saved file:  27830
saved file:  27831
saved file:  27832
saved file:  27833
saved file:  27834
saved file:  27835
saved file:  27836
saved file:  27837
save

In [ ]:
# validate saved features and calculate max length of all questions
for pair in kns:
  try:
    ques_id = pair[0]
    emb = np.load('/content/data/knowledge_embeddings/'+str(ques_id)+'.npy')
    # length = emb.shape[0]
    print(ques_id,'shape is', emb.shape)        
  except:
    None

流式输出内容被截断，只能显示最后 5000 行内容。
27786 shape is (52, 1024)
27787 shape is (7, 1024)
27788 shape is (14, 1024)
27789 shape is (33, 1024)
27790 shape is (2, 1024)
27791 shape is (14, 1024)
27792 shape is (14, 1024)
27793 shape is (49, 1024)
27794 shape is (7, 1024)
27795 shape is (14, 1024)
27796 shape is (25, 1024)
27797 shape is (2, 1024)
27798 shape is (14, 1024)
27799 shape is (36, 1024)
27800 shape is (43, 1024)
27801 shape is (7, 1024)
27802 shape is (14, 1024)
27803 shape is (15, 1024)
27804 shape is (2, 1024)
27805 shape is (14, 1024)
27806 shape is (47, 1024)
27807 shape is (60, 1024)
27808 shape is (7, 1024)
27809 shape is (14, 1024)
27810 shape is (15, 1024)
27811 shape is (2, 1024)
27812 shape is (14, 1024)
27813 shape is (36, 1024)
27814 shape is (28, 1024)
27815 shape is (7, 1024)
27816 shape is (14, 1024)
27817 shape is (13, 1024)
27818 shape is (2, 1024)
27819 shape is (28, 1024)
27820 shape is (35, 1024)
27821 shape is (7, 1024)
27822 shape is (14, 1024)
27823 shape is (17, 10

In [ ]:
# # What are positively charged,  thus allowing the compaction of the negatively charged DNA?
# # what are positively charged thus allowing the compaction of the negatively charged dna
# # [CLS] what are positively charged thus allowing the compaction of the negatively charged dna [SEP]
# # ['[CLS]', 'what', 'are', 'positively', 'charged', 'thus', 'allowing', 'the', 'compact', '##ion', 'of', 'the', 'negatively', 'charged', 'd', '##na', '[SEP]']
# # [101, 1184, 1132, 14257, 4601, 2456, 3525, 1103, 10114, 1988, 1104, 1103, 21598, 4601, 173, 1605, 102]
# # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# # tensor([[  101,  1184,  1132, 14257,  4601,  2456,  3525,  1103, 10114,  1988,  1104,  1103, 21598,  4601,   173,  1605,   102]])
# # tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


# for qa in data:
#   ques_id = qa['Question_Id']
#   ques = qa['Questions']    

#   # extract verbs and nouns from question
#   queries = ectract_query(ques)
#   # retrieve knowledge from knowledge base
#   # num: len(queries) * 5 
#   kns = knowledge_retrieval(queries)

#   kn_concat = '[CLS]'
#   for k in kns:
#     marked_text1 = ' ' + k + " [SEP]"
#     kn_concat += marked_text1   
#   tokenized_text1 = tokenizer.tokenize(kn_concat)[:-1]  
#   indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)    
#   segments_ids1 = [1] * len(tokenized_text1)    
#   tokens_tensor1 = torch.tensor([indexed_tokens1])
#   segments_tensors1 = torch.tensor([segments_ids1])

#   # tuple - shape: 25 * (1,17,1024) - (batch size,sentence length,embedding size)
#   with torch.no_grad():
#       # Because we set 'output_hidden_states = True', the third item will be the
#       # hidden states from all layers.
#       outputs1 = model(tokens_tensor1, segments_tensors1)
#       hidden_states1 = outputs1[2]
#   # Concatenate the tensors for all layers.
#   # Use 'stack' to create a new dimension in the tensor.
#   # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
#   token_embeddings1 = torch.stack(hidden_states1, dim=0)
#   # Remove dimension 1, the "batches".
#   # shape: (25,17,1024) - (layers,sentence length,embedding size)
#   token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
#   # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
#   # shape: (17,25,1024) - (sentence length,layers,embedding size)
#   token_embeddings1 = token_embeddings1.permute(1, 0, 2)

#   token_vecs_sum1 = []

#   for token in token_embeddings1:
#       # Sum the vectors from the last four layers.
#       sum_vec1 = torch.sum(token[-4:], dim=0)
#       token_vecs_sum1.append(sum_vec1)
#   # shape: (17,1024)
#   token_vec = np.array([t.numpy() for t in token_vecs_sum1])
#   # kn_emds.append(token_vec)
#   # kn_emds = np.concatenate(kn_emds)
#   # Write .npy file to '/content/embedding/' folder.
#   np.save(os.path.join(path, str(ques_id)+'.npy'), token_vec)
#   print("saved file: ", ques_id)
# print("saved all")

# # # validate saved features and calculate max length of all questions
# # max_len = 0
# # for pair in data:
# #     ques_id = pair['Question_Id']
# #     emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
# #     length = emb.shape[0]
# #     if length >= max_len:
# #         max_len = length
# #         print('current max length is', max_len)

saved file:  1
saved file:  2
saved file:  3
saved file:  4
saved file:  5
saved file:  6
saved file:  7
saved file:  8
saved file:  9
saved file:  10
saved file:  11
saved file:  12
saved file:  13
saved file:  14
saved file:  15
saved file:  16
saved file:  17
saved file:  18
saved file:  19
saved file:  20
saved file:  21
saved file:  22
saved file:  23
saved file:  24
saved file:  25
saved file:  26
saved file:  27
saved file:  28
saved file:  29
saved file:  30
saved file:  31
saved file:  32
saved file:  33
saved file:  34
saved file:  35
saved file:  36
saved file:  37
saved file:  38
saved file:  39
saved file:  40
saved file:  41
saved file:  42
saved file:  43
saved file:  44
saved file:  45
saved file:  46
saved file:  47
saved file:  48
saved file:  49
saved file:  50
saved file:  51
saved file:  52
saved file:  53
saved file:  54
saved file:  55
saved file:  56
saved file:  57
saved file:  58
saved file:  59
saved file:  60
saved file:  61
saved file:  62
saved file:  63
s

RuntimeError: ignored

## BERT embeddings

In [ ]:
for qa in data:
    ques_id = qa['Question_Id']
    # What are positively charged,  thus allowing the compaction of the negatively charged DNA?
    ques = qa['Questions']
    # what are positively charged thus allowing the compaction of the negatively charged dna
    after = preprocess_sentence(ques)
  
    # [CLS] what are positively charged thus allowing the compaction of the negatively charged dna [SEP]
    marked_text1 = "[CLS] " + after + " [SEP]"
    # ['[CLS]', 'what', 'are', 'positively', 'charged', 'thus', 'allowing', 'the', 'compact', '##ion', 'of', 'the', 'negatively', 'charged', 'd', '##na', '[SEP]']
    tokenized_text1 = tokenizer.tokenize(marked_text1)
    # [101, 1184, 1132, 14257, 4601, 2456, 3525, 1103, 10114, 1988, 1104, 1103, 21598, 4601, 173, 1605, 102]
    indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    segments_ids1 = [1] * len(tokenized_text1)
    # tensor([[  101,  1184,  1132, 14257,  4601,  2456,  3525,  1103, 10114,  1988,  1104,  1103, 21598,  4601,   173,  1605,   102]])
    tokens_tensor1 = torch.tensor([indexed_tokens1])
    # tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
    segments_tensors1 = torch.tensor([segments_ids1])

    # tuple - shape: 25 * (1,17,1024) - (batch size,sentence length,embedding size)
    with torch.no_grad():
        # Because we set 'output_hidden_states = True', the third item will be the
        # hidden states from all layers.
        outputs1 = model(tokens_tensor1, segments_tensors1)
        hidden_states1 = outputs1[2]
    # Concatenate the tensors for all layers.
    # Use 'stack' to create a new dimension in the tensor.
    # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
    token_embeddings1 = torch.stack(hidden_states1, dim=0)
    # Remove dimension 1, the "batches".
    # shape: (25,17,1024) - (layers,sentence length,embedding size)
    token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
    # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
    # shape: (17,25,1024) - (sentence length,layers,embedding size)
    token_embeddings1 = token_embeddings1.permute(1, 0, 2)

    token_vecs_sum1 = []

    for token in token_embeddings1:
        # Sum the vectors from the last four layers.
        sum_vec1 = torch.sum(token[-4:], dim=0)
        token_vecs_sum1.append(sum_vec1)
    # shape: (17,1024)
    token_vec = np.array([t.numpy() for t in token_vecs_sum1])

    
#     # Write .npy file to '/content/embedding/' folder.
#     np.save(os.path.join(path, str(ques_id)+'.npy'), token_vec)
#     print("saved file: ", ques_id)
# # print("saved all")

# # validate saved features and calculate max length of all questions
# max_len = 0
# for pair in data:
#     ques_id = pair['Question_Id']
#     emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
#     length = emb.shape[0]
#     if length >= max_len:
#         max_len = length
#         print('current max length is', max_len)
    break

(17, 1024)


# Knowledge

## SenticNet

In [ ]:
pip install senticnet

     |████████████████████████████████| 51.9MB 142kB/s 


In [ ]:
from senticnet.senticnet import SenticNet

sn = SenticNet()
concept_info = sn.concept('love')
polarity_label = sn.polarity_label('love')
polarity_value = sn.polarity_value('love')
moodtags = sn.moodtags('love')
semantics = sn.semantics('love')
sentics = sn.sentics('love')

In [ ]:
print(concept_info)
print(polarity_label)
print(polarity_value)
print(moodtags)
print(semantics)
print(sentics)

{'polarity_label': 'positive', 'polarity_value': '0.83', 'sentics': {'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}, 'moodtags': ['#joy', '#eagerness'], 'semantics': ['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']}
positive
0.83
['#joy', '#eagerness']
['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']
{'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}


## ConceptNet

In [ ]:
pip install conceptnet-lite

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 2.3MB 6.2MB/s 
     |████████████████████████████████| 870kB 12.2MB/s 
  Created wheel for peewee: filename=peewee-3.14.4-cp37-cp37m-linux_x86_64.whl size=592992 sha256=8cd8bf9ec663042c9460c97d740cc036a591f3a5f76022641ec733d49abe6218
  Stored in directory: /root/.cache/pip/wheels/a0/ce/ff/732bd2a2729233b3a55bb5d8557d500f7be41bd21df3f01048
  Created wheel for lmdb: filename=lmdb-0.97-cp37-cp37m-linux_x86_64.whl size=219767 sha256=2f858ce7912fbc450bcd41a6f0157509aae35fb3247beb7f2376d650440052c6
  Stored in directory: /root/.cache/pip/wheels/7f/4d/44/b299a9b22d8eaaaa455304687fb6d4c02d9178c1530850e12a
Successfully built peewee lmdb
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99


In [ ]:
import conceptnet_lite

conceptnet_lite.connect("/path/to/conceptnet.db")

File not found: /path/to/conceptnet.db
Download compressed database


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[*] 0 bytes / 1.85 GB @ 0 bytes/s [------------------] [0.0%, 0s left]    [*] 2.5 MB / 1.85 GB @ 12.6 MB/s [------------------] [0.1%, 0s left]    [*] 20.4 MB / 1.85 GB @ 67.8 MB/s [------------------] [1.1%, 0s left]    [*] 37.8 MB / 1.85 GB @ 94.4 MB/s [------------------] [2.0%, 0s left]    [*] 55.5 MB / 1.85 GB @ 111.0 MB/s [------------------] [2.9%, 0s left]    [*] 73.3 MB / 1.85 GB @ 122.2 MB/s [------------------] [3.9%, 0s left]    [*] 91.2 MB / 1.85 GB @ 130.2 MB/s [------------------] [4.8%, 0s left]   

In [ ]:
from conceptnet_lite import Label, edges_for

def query_knowledge(q):
  try:
    d = {}
    for e in edges_for(Label.get(text=q, language='en').concepts, same_language=True):
      # print(e.start.text, "::", e.end.text, "|", e.relation.name, "|", e.etc['weight'])
      d[(e.start.text,e.relation.name,e.end.text)] = e.etc['weight']
      # print(e.etc['weight'])
    kn = []
    for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
      # if i == 5:
      #   break
      k = ' '.join(f[0])
      k = k.replace('_', ' ')
      kn.append(k)
      # only one knowledge
      break
    return kn
  except:
    return []


In [ ]:
print(query_knowledge('edematous'))

['edematous synonym dropsical', 'antiedematous derived from edematous', 'edematously derived from edematous', 'nonedematous derived from edematous', 'edemateous form of edematous']
[]
[]


In [ ]:
from conceptnet_lite import Label

cat_concepts = Label.get(text='charge', language='en').concepts  
for c in cat_concepts:
    print("    Concept URI:", c.uri)
    print("    Concept text:", c.text)
    print("    Concept language:", c.language.name)